# SAVE PICKLES?


In [6]:
SAVING = True

# IMPORTING DATA

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mpldates
import pandas as pd
from pathlib import Path

# Local imports
#------------------------------------------------
import WireDAQ.PandasPlus           # Make sure this import is after pandas
import WireDAQ.Constants as cst
import WireDAQ.NXCALS as nx
import WireDAQ.Parser as parser
# import WireDAQ.Efficiency as eff



# Creating NXCALS variable containers
#------------------------------------------------
wires     = {'B1': [nx.NXCALSWire(loc = loc) for loc in ['L1B1','L5B1']],
             'B2': [nx.NXCALSWire(loc = loc) for loc in ['R1B2','R5B2']]}
beams     = [nx.NXCALSBeam(name) for name in ['B1','B2']]
LHC       = nx.NXCALSLHC()
b_slots   = np.arange(3564)
#------------------------------------------------

# Locating data
# _default_path   = '/home/phbelang/002mount/'
_default_path   = '/eos/project/l/lhc-lumimod/LuminosityFollowUp/2023/rawdata/'


# Loading wire current for each fill
variables = ['HX:FILLN','HX:BMODE','HX:BETASTAR_IP1'] + [wire.I for wire in wires['B2']] + [beam.Intensity for beam in beams]
file_list = list(Path(_default_path).glob('*/HX:BMODE=STABLE/*.parquet'))




to_exclude = ['8782','8817','8821']
file_list = [f for f in file_list if str(f).split('FILLN=')[1][:4] not in to_exclude ]

df = parser.from_parquet(file=file_list, variables=variables)



>>> Loading nx2pd.py version of 24.10.2022 @ 03:17PM



# Filtering: wires on  

In [7]:

for _var in ['HX:BMODE','HX:FILLN',"HX:BETASTAR_IP1",beams[0].Intensity,beams[1].Intensity]:
    df[_var] = df[_var].ffill()

filtered = df.loc[  (df["HX:BMODE"] == 'STABLE') & 
                    (df["HX:BETASTAR_IP1"] == 30)&
                    (df[beams[0].Intensity] > 300*1e11)&
                    (df[beams[1].Intensity] > 300*1e11)]

# Looking at values of max current
results = filtered.groupby('HX:FILLN')[['HX:BETASTAR_IP1'] + [wire.I for wire in wires['B2']] + [beam.Intensity for beam in beams]].max().dropna()

# Keeping only wires off
current_culumns = [wire.I for wire in  wires['B2']]
relevant_idx    = results.index[(np.any(results[current_culumns]>50,axis=1))]

results = results.loc[relevant_idx]
results.index = results.index.astype(int)
results.sort_index(inplace=True)

display(results)
print(f'{len(results)} fills found')

if SAVING:
    results.to_pickle("filter_wires_on.pkl")

,HX:BETASTAR_IP1,RPMC.UL16.RBBCW.R1B2:I_MEAS,RPMC.UL557.RBBCW.R5B2:I_MEAS,LHC.BCTDC.A6R4.B1:BEAM_INTENSITY,LHC.BCTDC.A6R4.B2:BEAM_INTENSITY
HX:FILLN,,,,,
8686,30.0,350.019196,350.041412,4.438907e+13,4.723224e+13
8690,30.0,350.020203,350.039276,4.601346e+13,4.784817e+13
8695,30.0,350.017487,350.046051,1.059030e+14,1.121122e+14
8696,30.0,350.020782,350.043060,1.199964e+14,1.213327e+14
8701,30.0,350.014587,350.043091,1.156984e+14,1.216359e+14
8723,30.0,350.018433,350.042236,4.635041e+13,4.804950e+13
8725,30.0,350.013550,350.037964,1.163369e+14,1.216243e+14
8729,30.0,350.015137,350.041656,1.352318e+14,1.405833e+14
8730,30.0,350.020233,350.051941,1.356038e+14,1.424001e+14


19 fills found


# Filtering: wires off

In [8]:

for _var in ['HX:BMODE','HX:FILLN',"HX:BETASTAR_IP1",beams[0].Intensity,beams[1].Intensity]:
    df[_var] = df[_var].ffill()

filtered = df.loc[  (df["HX:BMODE"] == 'STABLE') & 
                    (df["HX:BETASTAR_IP1"] == 30)&
                    (df[beams[0].Intensity] > 300*1e11)&
                    (df[beams[1].Intensity] > 300*1e11)]

# Looking at values of max current
results = filtered.groupby('HX:FILLN')[['HX:BETASTAR_IP1'] + [wire.I for wire in wires['B2']] + [beam.Intensity for beam in beams]].max().dropna()

# Keeping only wires off
current_culumns = [wire.I for wire in wires['B2']]
relevant_idx    = results.index[np.invert(np.any(results[current_culumns]>50,axis=1))]

results = results.loc[relevant_idx]
results.index = results.index.astype(int)
results.sort_index(inplace=True)

display(results)
print(f'{len(results)} fills found')

if SAVING:
    results.to_pickle("filter_wires_off.pkl")

,HX:BETASTAR_IP1,RPMC.UL16.RBBCW.R1B2:I_MEAS,RPMC.UL557.RBBCW.R5B2:I_MEAS,LHC.BCTDC.A6R4.B1:BEAM_INTENSITY,LHC.BCTDC.A6R4.B2:BEAM_INTENSITY
HX:FILLN,,,,,
8675,30.0,0.014525,0.041281,4.748497e+13,4.971151e+13
8685,30.0,0.016420,0.039100,4.942503e+13,5.093354e+13
8724,30.0,-0.001548,0.004208,1.451825e+14,1.472934e+14
8731,30.0,0.012387,0.041009,2.188798e+14,2.287248e+14
8741,30.0,0.014297,0.040100,2.485238e+14,2.513300e+14
8804,30.0,0.013705,0.033221,2.793173e+14,2.910900e+14
8850,30.0,0.012052,0.033797,7.907368e+13,8.139559e+13
8853,30.0,0.017679,0.036160,1.371002e+14,1.372801e+14


8 fills found


# Quality check: negative current?



In [5]:

for _var in ['HX:BMODE','HX:FILLN',"HX:BETASTAR_IP1",beams[0].Intensity,beams[1].Intensity]:
    df[_var] = df[_var].ffill()

filtered = df.loc[  (df["HX:BMODE"] == 'STABLE') &
                    (df['HX:FILLN'].astype(float) >= 8033) & 
                    (df["HX:BETASTAR_IP1"] == 30)&
                    (df[beams[0].Intensity] > 300*1e11)&
                    (df[beams[1].Intensity] > 300*1e11)]

# Looking at values of max current
results = filtered.groupby('HX:FILLN')[['HX:BETASTAR_IP1'] + [wire.I for wire in wires['B2']] + [beam.Intensity for beam in beams]].max().dropna()

# Keeping only wires off
current_culumns = [wire.I for wire in wires['B2']]
relevant_idx    = results.index[(np.any(results[current_culumns]<-10,axis=1))]

results = results.loc[relevant_idx]
results.index = results.index.astype(int)
results.sort_index(inplace=True)

display(results)
print(f'{len(results)} fills found')


,HX:BETASTAR_IP1,RPMC.UL16.RBBCW.R1B2:I_MEAS,RPMC.UL557.RBBCW.R5B2:I_MEAS,LHC.BCTDC.A6R4.B1:BEAM_INTENSITY,LHC.BCTDC.A6R4.B2:BEAM_INTENSITY
HX:FILLN,,,,,


0 fills found
